# Vulcões em Vênus

Nesse notebook iremos abordar uma solução para o problema [*Volcanoes on Venus* do Kaggle](https://www.kaggle.com/fmena14/volcanoesvenus). O dataset consiste em imagens de sonda do planeta Vênus e o problema consiste em predizer se uma determinada imagem contém pelo menos um vulcão ou não. 

#### Bibliotecas e ferramentas

In [34]:
import pandas as pd
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

#### Ler dados de treino e teste
Os dados já estão separados, em treino e teste, entre imagens e *labels* (se a respectiva imagem contém pelo menos um vulcão ou não). Observe que para os *labels* estamos usando apenas a primeira coluna do dataset, pois as demais não são importantes para o problema. Após a leitura os dados são normalizados usando o Scaler.

In [41]:
X_train = pd.read_csv("data/train_images.csv", header=None)
y_train = pd.read_csv("data/train_labels.csv")["Volcano?"]
X_test = pd.read_csv("data/test_images.csv", header=None)
y_test = pd.read_csv("data/test_labels.csv")["Volcano?"]

In [45]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)

#### Análise de Componentes Principais
Para reduzir a dimensionalidade das imagens (110 x110), usaremos PCA. Usamos **2000** componentes porque os primeiros **700** autovetores retém **~96%** da variância.

In [46]:
pca = PCA(n_components=2000)
pca.fit(X_train)
X_train_pca = pca.transform(X_train)
X_test_pca = pca.transform(X_test)

### Treinamento com SVM
Agora usamos os dados transformados para treinar e testar o modelo, com o método SVM.

In [71]:
svm = SVC(kernel='rbf', C=2.0)
svm.fit(X_train_pca, y_train)

SVC(C=2.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

### Testes
Resultados das predições no conjunto de testes.

In [72]:
pred = svm.predict(X_test_pca)
print(classification_report(y_test, pred))
print("Score: {}".format(accuracy_score(y_test, pred)))

             precision    recall  f1-score   support

          0       0.94      0.99      0.96      2300
          1       0.91      0.68      0.78       434

avg / total       0.94      0.94      0.94      2734

Score: 0.9389173372348207
